In [ ]:
import glob
import os
import re
import datetime
import subprocess
import shlex
import gdal
from importlib import reload
from subprocess import PIPE

import downloader
import af_import
import rst_import_lct
import rst_import_vcf
import rst_import
import polygon_import
import run_step1
import run_step2
import export_shp

## ActiveFire data

In [ ]:
# tag to identify active fire dataset
tag_af = 'mod_global_2016_v7m'

# source dir
ddir_af = 'downloads/firms'

# shp file names
arcnames = ['M6_28864', ]
af_fnames = [os.path.join(ddir_af, 'global_2016', 'fire_archive_%s.shp' % _) for _ in arcnames]

In [ ]:
# unpack active fire zip file if needed
if not os.path.exists('firms/global_2016/fire_archive_M6_28864.shp'):
    if not os.path.exists('2016-global-DL_FIRE_M6_28864.zip'):
        subprocess.run(shlex.split('wget https://s3-us-west-2.amazonaws.com/earthlab-finn/2016-global-DL_FIRE_M6_28864.zip'))
    if not os.path.exists('downloads/firms/global_2016'): 
        os.makedirs('downloads/firms/global_2016')
    subprocess.run(shlex.split('unzip 2016-global-DL_FIRE_M6_28864.zip -d downloads/firms/global_2016'))

## Download MODIS land cover and vegetation fields data

In [ ]:
# year for raster
year = 2016
is_leap = (year % 4 == 0)

# tag to identify datasets
tag_lct = 'modlct_%d' % year
tag_vcf = 'modvcf_%d' % year
tag_regnum = 'regnum'

url_lct = 'https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/%d.01.01/' % year
url_vcf = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/%d.03.%02d/' % (year, 5 if is_leap else 6)

ddir_lct = './downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/%d.01.01' % year
ddir_vcf = './downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/%d.03.%02d/' % (year, 5 if is_leap else 6)

In [ ]:
downloader.download_all(url_lct)

In [ ]:
downloader.purge_corrupted(ddir_lct, url=url_lct)

In [ ]:
downloader.download_all(url_vcf)

In [ ]:
downloader.purge_corrupted(ddir_vcf, url=url_vcf)

## PostGIS database settings

In [ ]:
os.environ['PGDATABASE'] = 'gis'
os.environ['PGUSER'] = 'docker'
os.environ['PGPASSWORD'] = 'docker'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'

In [ ]:
# show info for the database
!psql postgres -c 'SELECT version();'
!pg_lsclusters

In [ ]:
# TODO i want to move this to Dockerfile somehow
# create plpython, needed only once for the database
try:
    p = subprocess.run(shlex.split("psql -d gis -c 'CREATE LANGUAGE plpython3u;'"), check=True, capture_output=True)
except subprocess.CalledProcessError as e:
    if 'already exists' in e.stderr.decode():
        pass

## Import land cover type

Get filenames of MCD12Q1 land cover type data

In [ ]:
# grab hdf file names
search_string = "%(ddir)s/MCD12Q1.A%(year)s001.h??v??.006.*.hdf" % dict(
        ddir = ddir_lct, year=year)
fnames_lct = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames_lct) )

In [ ]:
reload(rst_import)
rst_import.main(tag_lct, 'lct', fnames_lct, run_merge=False, run_resample=True, run_import=True)

## Import vegetation continuous fields

In [ ]:
# grab hdf file names
search_string = "%(ddir)s/MOD44B.A%(year)s065.h??v??.006.*.hdf" % dict(
        ddir = ddir_vcf, year=year)
fnames_vcf = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames_vcf) )

In [ ]:
reload(rst_import)
rst_import.main(tag_vcf, 'vcf', fnames_vcf)

## Import countries of the world shapefile

In [ ]:
if not os.path.exists('all_countries/All_Countries.shp'):
    subprocess.run(shlex.split('wget https://s3-us-west-2.amazonaws.com/earthlab-finn/All_Countries.zip'))
    subprocess.run(shlex.split('unzip All_Countries.zip -d all_countries'))
    subprocess.run(['rm', 'All_Countries.zip'])

In [ ]:
polygon_import.main('regnum')

## Import active fire data

In [ ]:
reload(af_import)
af_import.main(tag_af, af_fnames)
print()
for i,fn in enumerate(af_fnames):
    print(fn)
    p = subprocess.run(['psql', '-c', 'select count(*) from "af_%s".af_in_%d;' % (tag_af, i+1)], stdout=PIPE)
    print(p.stdout.decode())


## Running "step 1" grouping points

In [ ]:
# You can either set fist and last day of analysis by your self here,

# firstday = datetime.date(year, 1, 1)
# lastday = datetime.date(year+1, 1, 1)

# or use the period covered by the input active fire file (keep both variables to None)

firstday = None
lastday = None


In [ ]:
reload(run_step1)
run_step1.main(tag_af, firstday=firstday, lastday=lastday, ver='v7m', run_prep = True, run_work=True)

## Running "step 2"

In [ ]:
assert run_step2.ver == 'v8b'

rasters = [
        {'tag': tag_lct,
            'kind': 'thematic',
            'variable': 'lct'},
        {'tag': tag_vcf,
            'kind': 'continuous',
            'variables': ['tree', 'herb', 'bare'],
            },
        {'tag': tag_regnum,
            'kind': 'polygons',
            'variable_in': 'region_num',
            'variable': 'regnum',
            },
        ]

In [ ]:
run_step2.main(tag_af, rasters, firstday=firstday, lastday=lastday)

## Export the output

In [ ]:
odir = '.'
schema = 'af_' + tag_af
tblname = 'out_{0}_{1}_{2}'.format(tag_lct, tag_vcf, tag_regnum)
flds = ('v_lct', 'f_lct', 'v_tree', 'v_herb', 'v_bare', 'v_regnum')
tblname

In [ ]:
export_shp.main(odir, schema, tblname, flds)